## Init

In [ ]:
import datetime
import glob
import os
import pandas as pd
import seaborn
import subprocess

PCAP_DIR = "../cap/"

In [ ]:
%pylab inline

In [ ]:
def read_pcap(filename, fields=[], display_filter="", 
              timeseries=False, strict=False):
    """ Read PCAP file into Pandas DataFrame object. 
    Uses tshark command-line tool from Wireshark.

    filename:       Name or full path of the PCAP file to read
    fields:         List of fields to include as columns
    display_filter: Additional filter to restrict frames
    strict:         Only include frames that contain all given fields 
                    (Default: false)
    timeseries:     Create DatetimeIndex from frame.time_epoch 
                    (Default: false)

    Syntax for fields and display_filter is specified in
    Wireshark's Display Filter Reference:
 
      http://www.wireshark.org/docs/dfref/
    """
    if timeseries:
        fields = ["frame.time_epoch"] + fields
    fieldspec = " ".join("-e %s" % f for f in fields)

    display_filters = fields if strict else []
    if display_filter:
        display_filters.append(display_filter)
    filterspec = "-Y '%s'" % " and ".join(f for f in display_filters)

    options = "-r %s -n -T fields -Eheader=y" % filename
    cmd = "tshark %s %s %s" % (options, filterspec, fieldspec)
    proc = subprocess.Popen(cmd, shell = True, 
                                 stdout=subprocess.PIPE,
                                 stderr=subprocess.PIPE)
    if timeseries:
        df = pd.read_table(proc.stdout, 
                        index_col = "frame.time_epoch", 
                        parse_dates=True, 
                        date_parser=datetime.datetime.fromtimestamp)
    else:
        df = pd.read_table(proc.stdout)
    return df

In [ ]:
def path_to_filename(path):
    name = os.path.basename(path)
    return os.path.splitext(name)[0]

In [ ]:
def load_all(fields=[], display_filter="", 
              timeseries=False, strict=False):
    files = glob.iglob(PCAP_DIR+"*.pcap")
    return {path_to_filename(file): read_pcap(file, fields, display_filter, timeseries, strict)
            for file in files}

## Cargar los PCAPs

In [ ]:
tables = load_all(["tcp.stream", "ip.src", "ip.dst", "tcp.seq", "tcp.ack", "tcp.window_size", "tcp.len"])

In [ ]:
print(list(tables.keys()))

In [ ]:
print(tables["EXACTAS-UBA"])